In [1]:
afcs_present = True

In [2]:
from src.queues_dir import QueuesDirectory

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory()  

[ИНФО][QUEUES] создан каталог очередей


In [3]:
from src.sitl import SITL
from geopy import Point as GeoPoint

# координата текущего положения машинки
home = GeoPoint(latitude=59.939032, longitude=30.315827) 

# идентификатор машинки (аналог VIN)
car_id = "m1" 

sitl = SITL(queues_dir=queues_dir, position=home, car_id=car_id)

[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1


In [4]:
from multiprocessing import Queue
from src.communication_gateway import BaseCommunicationGateway
from src.config import CONTROL_SYSTEM_QUEUE_NAME
from src.event_types import Event

class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway класс для реализации логики взаимодействия
    с системой планирования заданий

    Работает в отдельном процессе, поэтому создаётся как наследник класса Process
    """
    def _send_mission_to_consumers(self):
        """ метод для отправки сообщения с маршрутным заданием в систему управления """
        
        # имена очередей блоков находятся в файле src/config.py
        # события нужно отправлять в соответствие с диаграммой информационных потоков
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME

        # события передаются в виде экземпляров класса Event, 
        # описание класса находится в файле src/event_types.py
        event = Event(source=BaseCommunicationGateway.event_source_name,
                      destination=control_q_name,
                      operation="set_mission", parameters=self._mission
                      )

        # поиск в каталоге нужной очереди (в данном случае - системы управления)
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        # отправка события в найденную очередь
        control_q.put(event)

In [5]:
communication_gateway = CommunicationGateway(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи


In [6]:
from src.config import SERVOS_QUEUE_NAME
from src.control_system import BaseControlSystem

class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = SERVOS_QUEUE_NAME # замените на правильное название очереди
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_speed", parameters=speed
                      ) # код создания сообщения со скоростью для приводов

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_direction", parameters=direction
                      ) # код создания сообщения с направлением для приводов

        servos_q.put(event_speed)
        servos_q.put(event_direction)       

In [7]:
# найдите подходящее имя очереди в файле src/config.py и добавьте в строчку ниже
from src.config import CONTROL_SYSTEM_QUEUE_NAME # импортируем имя очереди для приводов
from src.navigation_system import BaseNavigationSystem


class NavigationSystem(BaseNavigationSystem):
    """Класс навигационного блока"""
    def _send_position_to_consumers(self):        
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        event = Event(
            source=BaseNavigationSystem.event_source_name,
            destination=control_q_name,
            operation="position_update",
            parameters=self._position
        )
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        control_q.put(event)

In [8]:
control_system = ControlSystem(queues_dir=queues_dir)
navigation_system = NavigationSystem(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации


In [9]:
from src.servos import Servos

servos = Servos(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][SERVOS] создан компонент сервоприводов


In [10]:
from src.mission_planner import MissionPlanner

mission_planner = MissionPlanner(queues_dir=queues_dir)

[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий


In [11]:
from src.mission_type import Mission, GeoSpecificSpeedLimit

mission = Mission(home=home,
                waypoints=[GeoPoint(latitude=59.9386, longitude=30.3121),
                           GeoPoint(latitude=59.9386, longitude=30.3149),
                           GeoPoint(latitude=59.9421, longitude=30.3067)
                           ],
                speed_limits=[
                    GeoSpecificSpeedLimit(0, 30),
                    GeoSpecificSpeedLimit(1, 60)
                ],
                armed=True)

In [12]:
mission_planner.set_new_mission(mission=mission)

In [13]:
car_id = "m1"

In [14]:
from time import sleep
from src.config import LOG_INFO

# создаём основные блоки
sitl = SITL(queues_dir=queues_dir, position=home, car_id=car_id, log_level=LOG_INFO)        # <- симулятор движения
communication_gateway = CommunicationGateway(queues_dir=queues_dir, log_level=LOG_INFO)     # <- Связь
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)                   # <- Система управления
navigation_system = NavigationSystem(queues_dir=queues_dir, log_level=LOG_INFO)             # <- Навигация
servos = Servos(queues_dir=queues_dir, log_level=LOG_INFO)                                  # <- Приводы


# запускаем созданные выше блоки системы, после этого они начнут обработку входящих событий
sitl.start()
navigation_system.start()
servos.start()
communication_gateway.start()
control_system.start()
mission_planner.start()


# пусть машинка немного поездит, 
# параметр sleep - время в секундах
sleep(5)

# останавливаем все компоненты
control_system.stop()
communication_gateway.stop()
mission_planner.stop()
sitl.stop()
servos.stop()
navigation_system.stop()

# дождёмся завершения работы всех компонентов
control_system.join()
communication_gateway.join()
mission_planner.join()
sitl.join()
servos.join()
navigation_system.join()

# подчистим все ресурсы для возможности повторного запуска в следующих модулях
del control_system, communication_gateway, mission_planner, sitl, servos, navigation_system

[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][SERVOS] создан компонент сервоприводов
[ИНФО][SITL] [SITL] старт симуляции[ИНФО][NAVIGATION] старт навигации
[ИНФО][SERVOS] старт блока приводов

[ИНФО][CONTROL] старт системы управления[ИНФО][COMMUNICATION] старт системы планирования заданий
[ИНФО][MISSION PLANNER] старт системы планирования заданий

[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][COMMUNICATION] получен новый маршрут, отправляем в получателям
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее

In [15]:
from src.config import CARGO_BAY_QUEUE_NAME, SERVOS_QUEUE_NAME
    
class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """

    def _send_speed_and_direction_to_consumers(self, speed, direction):
        servos_q_name = SERVOS_QUEUE_NAME # замените на правильное название очереди
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с желаемой скоростью
        event_speed = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_speed", parameters=speed
                      ) # код создания сообщения со скоростью для приводов

        # отправка сообщения с желаемым направлением
        event_direction = Event(source=BaseControlSystem.event_source_name,
                      destination=servos_q_name,
                      operation="set_direction", parameters=direction
                      ) # код создания сообщения с направлением для приводов

        servos_q.put(event_speed)
        servos_q.put(event_direction)   

    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation="lock_cargo",
                      parameters=None
                      )
        cargo_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation="release_cargo",
                      parameters=None
                      )
        cargo_q.put(event)

In [16]:
from time import sleep
from src.mission_planner import MissionPlanner
from src.cargo_bay import CargoBay
from src.config import LOG_ERROR, LOG_INFO
from src.mission_type import Mission, GeoSpecificSpeedLimit
from src.system_wrapper import SystemComponentsContainer

# вспомогательные блоки для отправки данных в СУПА
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender


mission_sender = MissionSender(
    queues_dir=queues_dir, client_id=car_id, log_level=LOG_INFO)
telemetry_sender = TelemetrySender(
    queues_dir=queues_dir, client_id=car_id, log_level=LOG_INFO)

home = GeoPoint(latitude=59.9386, longitude=30.3121)

# сократим маршрут для ускорения процесса
mission = Mission(
    home=home,
    waypoints=[home, GeoPoint(latitude=59.9421, longitude=30.3067)
               ],
    speed_limits=[
        GeoSpecificSpeedLimit(0, 60)
    ],
    armed=True)

mission_planner = MissionPlanner(
    queues_dir=queues_dir, afcs_present=afcs_present)
mission_planner.set_new_mission(mission=mission)

sitl = SITL(queues_dir=queues_dir, position=home,
            car_id=car_id, log_level=LOG_INFO, post_telemetry=afcs_present)
communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_INFO)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)
navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_INFO)
servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)


# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        mission_planner
    ] if afcs_present else [
        sitl,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        mission_planner
    ])

system_components.start()

# пусть машинка немного поездит,
# параметр sleep - время в секундах
sleep(35)

# останавливаем все компоненты
system_components.stop()

# подчистим все ресурсы для возможности повторного запуска в следующих модулях
system_components.clean()

[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][SITL] симулятор создан, ID m1
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][COMMUNICATION] создан компонент связи
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][NAVIGATION] создан компонент навигации
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION_PLANNER.MQTT] старт клиента телеметрии
[ИНФО][SITL.MQTT] старт клиента телеметрии[ИНФО][MISSION_PLANNER.MQTT] клиент отправки маршрута создан и запущен

[ИНФО][SITL] [SITL] старт симуляции
[ИНФО][SITL.MQTT] клиент отправки телеметрии создан и запущен[ИНФО][NAVIGATION] старт нави